In [2]:
# 필요한 라이브러리를 불러옵니다.
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm.notebook import tqdm 

In [3]:
# User-Agent 정보를 저장합니다.
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# 검색어와 날짜 범위를 기반으로 URL을 생성하는 함수입니다.
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=1):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]


In [4]:
# 주어진 URL에서 네이버 뉴스 기사의 링크를 가져오는 함수입니다.
def get_naver_articles(urls):
    articles = []
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")
        articles += [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info") if "news.naver.com" in link.attrs['href']]
        time.sleep(1)
    return articles


In [7]:
# 주어진 뉴스 기사 URL에서 제목, 내용, 날짜를 크롤링하는 함수입니다.
def get_article_content(article_url):
    news = requests.get(article_url, headers=headers)
    soup = BeautifulSoup(news.text, "html.parser")

    title = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2") or soup.select_one("#content > div.end_ct > div > h2")
    content = soup.select("#dic_area") or soup.select("#articeBody")
    
    cleaned_title = re.sub('<[^>]*>', '', str(title))
    cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')

    try:
        date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
    except AttributeError:
        date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))

    return date, cleaned_title, cleaned_content


In [11]:
# 기존의 main 함수를 수정하여 search, start_date, end_date를 반환하도록 합니다.
def main():
    search = input("검색할 키워드를 입력해주세요:")
    start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
    end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

    urls = makeUrl(search, start_date, end_date)
    article_urls = get_naver_articles(urls)
    
    dates, titles, contents, links = [], [], [], []
    for url in tqdm(article_urls):
        date, title, content = get_article_content(url)
        dates.append(date)
        titles.append(title)
        contents.append(content)
        links.append(url)
    
    df = pd.DataFrame({'date': dates, 'title': titles, 'link': links, 'content': contents})
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    display(df)  # Jupyter Notebook에서 DataFrame을 보기 좋게 출력하기 위해 display 함수를 사용합니다.
    
    return search, start_date, end_date, df

# main 함수를 실행하고 반환된 값을 변수에 할당합니다.
search, start_date, end_date, df = main()


,date,title,link,content
0,2017-01-01 12:03:13,[2017 국제 핫이슈]블록체인 기술 확산,https://n.news.naver.com/mnews/article/030/000...,\n\t\t\t블록체인은 `제2의 인터넷` `넥스트 인터넷`으로 불린다. 새해에는 ...
1,2017-01-01 16:03:04,"[신년기획]블록체인, 금융권 넘어 IT기업도 관심",https://n.news.naver.com/mnews/article/030/000...,\n\t\t\t“블록체인은 인터넷 이후 나온 가장 혁신적이며 파괴적인 기술이다.”블...
2,2017-01-02 17:04:01,블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자,https://n.news.naver.com/mnews/article/029/000...,\n올 세계 은행 80% '블록체인' 도입 전망금융회사 거래 비용 약 30% 절감 ...
3,2017-01-03 07:28:03,"비트코인, 3년만에 처음으로 1000달러 돌파",https://n.news.naver.com/mnews/article/018/000...,\n\t\t\t[이데일리 김경민 기자] 디지털 화폐인 비트코인 가격이 3년 만에 처...
4,2017-01-03 10:49:02,"'암호화 화폐' 비트코인, 3년만에 1천달러 돌파",https://n.news.naver.com/mnews/article/092/000...,\n\t\t\t(지디넷코리아=손경호 기자)전 세계에서 가장 많이 쓰이고 있는 암호화...
5,2017-01-03 16:48:03,"KISA, 암호기술팀·블록체인TF 신설",https://n.news.naver.com/mnews/article/031/000...,\n\t\t\t&lt;아이뉴스24&gt;[아이뉴스24 성지은기자] 한국인터넷진흥원(...


In [12]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
df.to_csv(filename, encoding='utf-8-sig', index=False)